In [449]:
!python/detect.py --crop_mode=selective_search --pretrained_model=models/bvlc_reference_rcnn_ilsvrc13/bvlc_reference_rcnn_ilsvrc13.caffemodel --model_def=models/bvlc_reference_rcnn_ilsvrc13/deploy.prototxt --raw_scale=255 _temp/det_input.txt _temp/det_output.h5


CPU mode
I0530 12:11:33.342443 19592 net.cpp:49] Initializing net from parameters: 
name: "R-CNN-ilsvrc13"
state {
  phase: TEST
}
layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 10
      dim: 3
      dim: 227
      dim: 227
    }
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "norm1"
  type: "LRN"
  bottom: "pool1"
  top: "norm1"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "norm1"
  top: "conv2"
  convolution_param {
    num_output: 256
    pad: 2
    kernel_size: 5
    group: 2
  }
}
layer {
  name: "relu2"
  ty

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [450]:
df = pd.read_hdf('_temp/det_output.h5', 'df')
print(df.shape)
print(df.iloc[0])


(1200, 5)
prediction    [-2.9569, -3.29594, -2.4833, -2.57037, -2.5282...
ymin                                                         15
xmin                                                        420
ymax                                                      956.5
xmax                                                      749.5
Name: /root/caffe/images/example_17_1.jpg, dtype: object


In [10]:
with open('data/ilsvrc12/det_synset_words.txt') as f:
    labels_df = pd.DataFrame([
        {
            'synset_id': l.strip().split(' ')[0],
            'name': ' '.join(l.strip().split(' ')[1:]).split(',')[0]
        }
        for l in f.readlines()
    ])
labels_df.sort('name')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,name,synset_id
0,accordion,n02672831
1,airplane,n02691156
2,ant,n02219486
3,antelope,n02419796
4,apple,n07739125
5,armadillo,n02454379
6,artichoke,n07718747
7,axe,n02764044
8,baby bed,n02766320
9,backpack,n02769748


In [451]:
predictions_df = pd.DataFrame(np.vstack(df.prediction.values), columns=labels_df['name'])

In [467]:
max_s = predictions_df.max(0)
max_s.sort(ascending=False)
print(max_s[:10])

name
person            2.502860
maillot           0.703431
miniskirt        -0.361568
chair            -0.450376
stethoscope      -0.529532
face powder      -0.551121
computer mouse   -0.703258
perfume          -0.710759
ladle            -0.864091
guitar           -0.896559
dtype: float32


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  from ipykernel import kernelapp as app


In [466]:
i = predictions_df['person'].argmax()

f = pd.Series(df['prediction'].iloc[i], index=labels_df['name'])

im = plt.imread('images/example_17_1.jpg')
#plt.imshow(im)
#currentAxis = plt.gca()

#det = df.iloc[i]


#coords = (det['xmin'], det['ymin']), det['xmax'] - det['xmin'], det['ymax'] - det['ymin']
#currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='r', linewidth=5))


In [468]:
def nms_detections(dets, overlap=0.7):
    """
    Non-maximum suppression: Greedily select high-scoring detections and
    skip detections that are significantly covered by a previously
    selected detection.

    This version is translated from Matlab code by Tomasz Malisiewicz,
    who sped up Pedro Felzenszwalb's code.
    """
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    ind = np.argsort(dets[:, 4])

    w = x2 - x1
    h = y2 - y1
    area = (w * h).astype(float)

    pick = []
    while len(ind) > 0:
        i = ind[-1]
        pick.append(i)
        ind = ind[:-1]

        xx1 = np.maximum(x1[i], x1[ind])
        yy1 = np.maximum(y1[i], y1[ind])
        xx2 = np.minimum(x2[i], x2[ind])
        yy2 = np.minimum(y2[i], y2[ind])

        w = np.maximum(0., xx2 - xx1)
        h = np.maximum(0., yy2 - yy1)

        wh = w * h
        o = wh / (area[i] + area[ind] - wh)

        ind = ind[np.nonzero(o <= overlap)[0]]

    return dets[pick, :]

In [469]:
scores = predictions_df['person']
windows = df[['xmin', 'ymin', 'xmax', 'ymax']].values
dets = np.hstack((windows, scores[:, np.newaxis]))
nms_dets = nms_detections(dets)

In [470]:
plt.imshow(im)
currentAxis = plt.gca()
colors = ['r', 'b', 'y','g']
for c, det in zip(colors, nms_dets[:4]):
    currentAxis.add_patch(
        plt.Rectangle((det[0], det[1]), det[2]-det[0], det[3]-det[1],
        fill=False, edgecolor=c, linewidth=5)
    )
print 'scores:', nms_dets[:4, 4]

scores: [ 2.50286031  2.38412046  1.40518451  1.39831328]


In [480]:
if nms_dets[0 ,4]>1 and (nms_dets[1 ,4]+nms_dets[2 ,4]+nms_dets[3 ,4])/2<nms_dets[0, 4]:
    plt.imshow(im)
    currentAxis = plt.gca()
    det = df.iloc[i]
    coords = (det['xmin'], det['ymin']), det['xmax'] - det['xmin'], det['ymax'] - det['ymin']
    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='r', linewidth=5))
    
elif nms_dets[0 ,4]<=1 and nms_dets[0 ,4]>0 and (nms_dets[1 ,4]+nms_dets[2 ,4]+nms_dets[3 ,4])/2<nms_dets[0, 4]:
    plt.imshow(im)
    currentAxis = plt.gca()
    currentAxis.add_patch(
        plt.Rectangle((min(nms_dets[0][0],nms_dets[1][0],nms_dets[2][0],nms_dets[3][0]), min(nms_dets[0][1],nms_dets[1][1],nms_dets[2][1],nms_dets[3][1])), max(nms_dets[0][2],nms_dets[1][2],nms_dets[2][2],nms_dets[3][2])-min(nms_dets[0][0],nms_dets[1][0],nms_dets[2][0],nms_dets[3][0]), max(nms_dets[0][3],nms_dets[1][3],nms_dets[2][3],nms_dets[3][3])-min(nms_dets[0][1],nms_dets[1][1],nms_dets[2][1],nms_dets[3][1]),
        fill=False, edgecolor='r', linewidth=5)
    )
    
else:
    plt.imshow(im)
    currentAxis = plt.gca()
    currentAxis.add_patch(
        plt.Rectangle(((nms_dets[0][0]+nms_dets[1][0]+nms_dets[2][0]+nms_dets[3][0])/4, (nms_dets[0][1]+nms_dets[1][1]+nms_dets[2][1]+nms_dets[3][1])/4), (nms_dets[0][2]+nms_dets[1][2]+nms_dets[2][2]+nms_dets[3][2])/4-(nms_dets[0][0]+nms_dets[1][0]+nms_dets[2][0]+nms_dets[3][0])/4, (nms_dets[0][3]+nms_dets[1][3]+nms_dets[2][3]+nms_dets[3][3])/4-(nms_dets[0][1]+nms_dets[1][1]+nms_dets[2][1]+nms_dets[3][1])/4,
        fill=False, edgecolor='r', linewidth=5)
    )
    
    
    

In [481]:
plt.show(im.all)